In [31]:
# 한빛출판네트워크 - 새로나온 책

In [32]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd 

In [33]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/store/books/new_book_list.html'
url = base_url + sub_url
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [34]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [35]:
li = lis[0]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B6694310162'

In [36]:
res = requests.get(base_url + href)
book_soup = BeautifulSoup(res.text, 'html.parser')

In [37]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'하루 한 장 초등과학 365'

In [38]:
book_lis = info.select('.info_list > li')
author = book_lis[0].find('span').get_text().strip()
author

'치바 가즈요시'

In [39]:
if len(book_lis) == 5:
    translator = '-'
else:
    translator = book_lis[1].find('span').get_text().strip()
translator

'허영은 , 유우종 (감수)'

In [40]:
date = book_lis[-4].find('span').get_text().strip()
date

'2021-12-14'

In [41]:
page = book_lis[-3].find('span').get_text().strip().split()[0]
page = int(page)
page

416

In [42]:
payments = book_soup.select_one('.payment_box.curr').find_all('p')
price = payments[0].select_one('.pbr').get_text().strip()
price

'19,800원'

In [43]:
sales = payments[1].select_one('.pbr').get_text().split('(')[0]
sales

'17,820원'

In [44]:
mileage = payments[2].select_one('.pbr').get_text().split()[0]
mileage

'990점'

In [46]:
lines = []
for li in lis:
    book_url = li.find('a')['href']
    res = requests.get(base_url + book_url)
    book_soup = BeautifulSoup(res.text, 'html.parser')

    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    book_lis = info.select('.info_list > li')
    author = book_lis[0].find('span').get_text().strip()
    if len(book_lis) == 5:
        translator = '  '
    else:
        translator = book_lis[1].find('span').get_text().strip()
    date = book_lis[-4].find('span').get_text().strip()
    page = book_lis[-3].find('span').get_text().strip().split()[0]
    
    payments = book_soup.select_one('.payment_box.curr').find_all('p')
    price = payments[0].select_one('.pbr').get_text().strip()
    sales = payments[1].select_one('.pbr').get_text().split('(')[0]
    mileage = payments[2].select_one('.pbr').get_text().split()[0]

    lines.append([title,author,translator,date,page,price,sales,mileage])

df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','정가','판매가','마일리지'])
df.tail()

,제목,저자,번역,출간일,페이지,정가,판매가,마일리지
15,나도 영문 손글씨 잘 쓰면 소원이 없겠네,윤정희(리제 캘리그라피),,2021-11-05,240,"16,800원","15,120원",840점
16,구글 BERT의 정석,수다르산 라비찬디란,"전희원 , 정승환 , 김형준",2021-11-03,368,"34,000원","30,600원","1,700점"
17,1000개 숨은그림찾기 몬스터 : 찾아도 찾아도 끝판왕,김확실,,2021-11-02,96,"8,000원","7,200원",400점
18,1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕,전진희,,2021-11-02,96,"8,000원","7,200원",400점
19,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,,2021-11-02,96,"8,000원","7,200원",400점


In [23]:
def convert_unit(s):
    return int(s.replace(',','').replace('원','').replace('점',''))

In [24]:
from tqdm.notebook import tqdm

lines = []
for page in tqdm(range(1,11)):
    url = f'{base_url}{sub_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    
    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(res.text, 'html.parser')

        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        author = book_lis[0].find('span').get_text().strip()
        if len(book_lis) == 5:
            translator = '-'
        else:
            translator = book_lis[1].find('span').get_text().strip()
        date = book_lis[-4].find('span').get_text().strip()
        page = book_lis[-3].find('span').get_text().strip().split()[0]
        
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        price = payments[0].select_one('.pbr').get_text().strip()
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        mileage = payments[2].select_one('.pbr').get_text().split()[0]

        lines.append([title,author,translator,date,int(page),
                      convert_unit(price),convert_unit(sales),convert_unit(mileage)])

df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','정가','판매가','마일리지'])
df.tail()

  0%|          | 0/10 [00:00<?, ?it/s]

,제목,저자,번역,출간일,페이지,정가,판매가,마일리지
195,"IT CookBook, C 언어 일취월장(2판)",신윤환,2021-02-24,800 쪽,9791156645313,28000,28000,840
196,소비자행동,김학윤,-,2021-02-23,660,35000,35000,1050
197,재미있고 빠른 어휘력 동화 – 두근두근 이 마음은 뭘까,"김세실 , 김도윤(그림)",-,2021-02-20,104,12000,10800,600
198,재미있고 빠른 어휘력 동화 – 키득키득 재밌는 의성어 의태어,"김예실 , 김희선(그림)",-,2021-02-20,104,12000,10800,600
199,웹 애플리케이션 보안,앤드루 호프먼,최용,2021-02-19,372,30000,27000,1500


In [25]:
len(lis)

20

In [26]:
lis[15]

<li class="sub_book_list">
<div class="view_box">
<!--
            <span class="view_box_block" onclick="location.href='/store/books/look.php?p_code=B9718869266';">
            -->
<span class="view_box_block">
<span class="img_hidden_a">
<span class="pop_quick_menu">
<a href="/store/books/look.php?p_code=B9718869266"><img alt="상세보기" src="https://www.hanbit.co.kr/images/common/icon_view_detail.png"/></a>
</span>
<span class="price">28,000<span>원</span></span>
<span class="pop_quick_bg" onclick="location='/store/books/look.php?p_code=B9718869266'"></span>
</span>
<!-- 예약도서인 경우 -->
<!-- ebook이 도서인 경우 -->
<img alt="" class="thumb" src="/data/books/B9718869266_m.jpg"/>
</span>
<div class="info">
<p class="book_tit"><a href="/store/books/look.php?p_code=B9718869266">IT CookBook, C 언어 일취월장(2판)</a></p>
<p class="book_writer">신윤환 </p>
</div>
</div>
</li>

In [27]:
href = lis[15].find('a')['href']
res = requests.get(base_url + href)
book_soup = BeautifulSoup(res.text, 'html.parser')

In [28]:
book_lis = info.select('.info_list > li')
book_dict = {}
for li in book_lis:
    items = li.get_text().split(':')
    book_dict[items[0].strip()] = items[1].strip()
    
book_dict

{'저자': '앤드루 호프먼',
 '번역': '최용',
 '출간': '2021-02-19',
 '페이지': '372 쪽',
 'ISBN': '9791162243930',
 '물류코드': '10393'}

In [29]:
lines = []
for page in tqdm(range(1,11)):
    url = f'{base_url}{sub_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    
    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(res.text, 'html.parser')

        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        book_dict = {}
        for li in book_lis:
            items = li.get_text().split(':')
            book_dict[items[0].strip()] = items[1].strip()
        author = book_dict['저자']
        translator = book_dict['번역'] if '번역' in book_dict.keys() else '-' 
        date = book_dict['출간']
        page = book_dict['페이지'].split()[0]
        
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        price = payments[0].select_one('.pbr').get_text().strip()
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        mileage = payments[2].select_one('.pbr').get_text().split()[0]

        lines.append([title,author,translator,date,int(page),
                      convert_unit(price),convert_unit(sales),convert_unit(mileage)])

df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','정가','판매가','마일리지'])
df.tail()

  0%|          | 0/10 [00:00<?, ?it/s]

,제목,저자,번역,출간일,페이지,정가,판매가,마일리지
195,"IT CookBook, C 언어 일취월장(2판)",신윤환,-,2021-02-24,800,28000,28000,840
196,소비자행동,김학윤,-,2021-02-23,660,35000,35000,1050
197,재미있고 빠른 어휘력 동화 – 두근두근 이 마음은 뭘까,"김세실 , 김도윤(그림)",-,2021-02-20,104,12000,10800,600
198,재미있고 빠른 어휘력 동화 – 키득키득 재밌는 의성어 의태어,"김예실 , 김희선(그림)",-,2021-02-20,104,12000,10800,600
199,웹 애플리케이션 보안,앤드루 호프먼,최용,2021-02-19,372,30000,27000,1500


In [45]:
df.to_csv('한빛도서.csv', index=False)